In [5]:
from models import dc_D, dc_G, dc_d, dc_g, GoodDiscriminator, GoodGenerator, GoodDiscriminatord
from VisionData import VisionData
import torch
from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
torch.backends.cudnn.benchmark = True

# CIFAR 10

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
learning_rate = 0.0001
batch_size = 64
z_dim = 128
dropout = None
# Dropout: None or 0.5
if dropout is not None:
    print('dropout!')
    D = GoodDiscriminatord(dropout=dropout)
else:
    D = GoodDiscriminator()
G = GoodGenerator()
dataset = CIFAR10(root='datas/cifar10', download=True, train=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
]))
trainer = VisionData(D=D, G=G, device=device, dataset=dataset, z_dim=z_dim, batchsize=batch_size, 
                      lr=learning_rate, show_iter=500, weight_decay=0.0, d_penalty=0.0, g_penalty=0, 
                      noise_shape=(64, z_dim), gp_weight=0, gpu_num=2)
trainer.train_bcgd(epoch_num=600, mode='MCGD', collect_info=False, dataname='CIFAR10-WGAN', 
                   logname='CIFAR10-WGAN', loss_type='WGAN')
# loss_type : WGAN, JSD

cpu
Files already downloaded and verified
learning rate: 0.00010 
weight decay: 0.00000
l2 penalty on discriminator: 0.00000
l2 penalty on generator: 0.00000
gradient penalty weight: 0.00


KeyboardInterrupt: 

In [6]:
torch.cuda.is_available()

False